# Experiments

In [ ]:
# Utility Code
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
import numpy as np

# Check palettes here: https://www.practicalpythonfordatascience.com/ap_seaborn_palette

landlocked_palette = sns.color_palette("Oranges_r", 4)
coastal_palette = sns.color_palette("Greens_r", 4)
polar_palette = sns.color_palette("Blues_r", 4)
equatorial_palette = sns.color_palette("Purples_r", 4)
intraorbital_palette = sns.color_palette("Reds_r", 4)
transorbital_palette = sns.color_palette("Wistia_r", 4)

cases = {
    "Landlocked": landlocked_palette,
    "Coastal": coastal_palette,
    "Polar": polar_palette,
    "Equatorial": equatorial_palette,
    "IntraOrbital": intraorbital_palette,
    "TransOrbital": transorbital_palette,
}

directions = ["East", "West", "North", "South"]


def load_data(path: str):
    """Load CSV data."""
    if not os.path.isfile(path):
        raise Exception(f"The path {path} does not exist.")
    return pd.read_csv(path)

def load_json_data(path: str):
    """Load JSON data."""
    if not os.path.isfile(path):
        raise Exception(f"The path {path} does not exist.")
    with open(path) as file:
        data = json.load(file)
    print(data)
    return pd.DataFrame(data['latencies'])
    # return pd.json_normalize(data)
    
    
    # return pd.read_json(path)


def graph(plot_fn, ax, imax=1):
    """Plot the median route latency for each case and direction across frames.

    To enable RDiS, set imax to a value greater than 1.
    """
    defence = ""
    if imax != 1:
        defence = f"{imax}"
    else:
        defence = "OFF"
    for case, palette in cases.items():
        for idx, direction in enumerate(directions):
            plot_fn(case, direction, defence, palette, idx, ax)

## 1. Network Performance relative to $i_\text{max}$
Plots the network performance relative to $i_\text{max}$ for $i_\text{max} \in \{3, 6, 9\}$.

In [ ]:
def plot_median_latency(case, direction, defence, palette, idx, ax):
    filename = f"{case}_{direction}_{defence}"
    data = load_json_data(f"Logs/Captures/{filename}/rtt.json")
    data["median"] = data["rtt"].dropna().apply(np.median)
    sns.lineplot(
        data=data,
        x="frame",
        y="median",
        color=palette[idx],
        label=f"{case} {direction}",
        ax=ax,
    )


fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Defence disabled
graph(plot_median_latency, axs[0, 0])

# imax = 3
graph(plot_median_latency, ax=axs[1, 0], imax=3)

# imax = 6
graph(plot_median_latency, ax=axs[2, 0], imax=6)

# imax = 9
graph(plot_median_latency, ax=axs[3, 0], imax=9)

for ax in axs:
    ax.set_xlabel("Snapshot")
    ax.set_ylabel("Median Latency (ms)")
    # ax.set_ylim(-.5, 20.5)  # TODO: CHANGE THIS.
    ax.tick_params(axis="y", labelcolor="black")
    ax.grid(True)

    # Remove the subplot if there are no lines.
    if len(ax.get_lines()) == 0:
        fig.delaxes(ax)

plt.tight_layout()
plt.legend()
plt.show()

## 2. Simulated Probability of Congestion relative to $i_\text{max}$
Plot the probability of congestion relative to $i_\text{max}$ using spare capacity data retrieved from the simulator for $i_\text{max} \in \{3, 6, 9\}$..

In [ ]:
def plot_probability_congestion(case, direction, defence, palette, idx):
    filename = f"{case}_{direction}_{defence}"
    data = load_data(f"{filename}/{filename}.csv")
    successes = data["FINAL CAPACITY"].value_counts()[0]
    probability = successes / len(data)
    sns.barplot(
        x="{case} {direction}",
        y=probability,
        color=palette[idx],
        ax=ax,
        label="{case} {direction}",
    )

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Defence disabled
graph(plot_probability_congestion, axs[0])

# imax = 3
graph(plot_probability_congestion, ax=axs[1], imax=3)

# imax = 6
graph(plot_probability_congestion, ax=axs[2], imax=6)

# imax = 9
graph(plot_probability_congestion, ax=axs[3], imax=9)


for ax in axs:
    # ax.set_xlabel("Snapshot")
    ax.set_ylabel("Probability of Congestion")
    # ax.set_ylim(-.5, 20.5)  # TODO: CHANGE THIS.
    # ax.tick_params(axis="y", labelcolor="black")
    ax.grid(True)

    # Remove the subplot if there is no data.
    if len(ax.get_lines()) == 0:
        fig.delaxes(ax)

plt.tight_layout()
plt.legend()
plt.show()

## 3. Simulated Expected Malicious Traffic relative to $i_\text{max}$

In [ ]:
def plot_expected_capacity(case, direction, defence, palette, idx):
    filename = f"{case}_{direction}_{defence}"
    data = load_data(f"{filename}/{filename}.csv")
    plot_data = pd.DataFrame(
        {
            "case direction": [f"{case} {direction}"],
            "mean": data.mean()["FINAL CAPACITY"],
            "stddev": data.std()["FINAL CAPACITY"],
        }
    )

    sns.barplot(
        x="case direction",
        y="mean",
        yerr="stddev",
        data=plot_data,
        color=palette[idx],
        ax=ax,
        label="{case} {direction}",
    )


fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Defence disabled
graph(plot_expected_capacity, axs[0])

# imax = 3
graph(plot_expected_capacity, ax=axs[1], imax=3)

# imax = 6
graph(plot_expected_capacity, ax=axs[2], imax=6)

# imax = 9
graph(plot_expected_capacity, ax=axs[3], imax=9)


for ax in axs:
    # ax.set_xlabel("Snapshot")
    ax.set_ylabel("Expected Spare Capacity")
    # ax.set_ylim(-.5, 20.5)  # TODO: CHANGE THIS.
    # ax.tick_params(axis="y", labelcolor="black")
    ax.grid(True)

    # Remove the subplot if there is no data.
    if len(ax.get_lines()) == 0:
        fig.delaxes(ax)

plt.tight_layout()
plt.legend()
plt.show()